In [93]:
import pandas as pd
from itertools import chain
import glob
import numpy as np
file = pd.concat([pd.read_csv(f, index_col=None,header=None) for f in glob.glob('C:\\Users\\User\\Documents\\datasets\\fin*.csv')])
file2  = pd.concat([pd.read_csv(f, index_col=None,header=None) for f in glob.glob('C:\\Users\\User\\Documents\\datasets\\final*.csv')])
file3  = pd.concat([pd.read_csv(f, index_col=None,header=None) for f in glob.glob('C:\\Users\\User\\Documents\\datasets\\leak*.csv')])
file4 = pd.read_csv('C:\\Users\\User\\Documents\\datasets\\fin100.csv')
file_full = pd.concat([file,file2,file3,file4])

In [94]:
df = pd.DataFrame(file_full[file_full.columns[0:]].apply(lambda x: ' '.join(x.astype(str)),axis=1))
df = pd.DataFrame(df[0].str.replace('date','dateday'))
#split each fight into a separate column
df_split = pd.DataFrame(df[0].str.split('date',expand=True))
def func(df,col):
    return df[col].str.extract('day(?P<day>.*?)firstBoxerRating(?P<firstBoxerRating>.*?)firstBoxerWeight(?P<firstBoxerWeight>.*?)judges(?P<JudgeID>.*?)links(?P<Links>.*?)location(?P<location>.*?)metadata(?P<metadata>.*?)numberOfRounds(?P<numberofrounds>.*?)outcome(?P<outcome>.*?)rating(?P<rating>.*?)referee(?P<referee>.*?)secondBoxer(?P<secondBoxer>.*?)secondBoxerLast6(?P<secondBoxerLast6>.*?)secondBoxerRating(?P<secondBoxerRating>.*?)secondBoxerRecord(?P<secondBoxerRecord>.*?)secondBoxerWeight(?P<secondBoxerWeight>.*?)titles(?P<titles>.*?){')
for i in range(1,len(df_split.columns)-1):
    df_split[['date'+str(i), 'firstBoxerRating'+str(i), 'firstBoxerWeight'+str(i), 'JudgeID'+str(i), 'Links'+str(i),
       'location'+str(i), 'metadata'+str(i), 'numberofrounds'+str(i), 'outcome'+str(i), 'rating'+str(i),
       'referee'+str(i), 'secondBoxer'+str(i), 'secondBoxerLast6'+str(i), 'secondBoxerRating'+str(i),
       'secondBoxerRecord'+str(i), 'secondBoxerWeight'+str(i), 'titles'+str(i)]] = func(df_split,i)

In [95]:
titles = ['titles'+str(i) for i in range(1,85)]
masked = df_split[titles].astype(str)
df_split['global_id']=masked.apply(','.join,axis=1).str.extract(r'(\d{6})', expand = False).fillna('')
# df_split['global_id'] = masked.apply(','.join,axis=1).str.extract(r'(\d{4})', expand = False).fillna('')

In [96]:
#get name,global id and other details and merge with dataset
new_file = pd.read_csv("C:\\Users\\User\\Documents\\GitHub\\SpringboardCapstoneBoxingPredictionWebApp\\boxingdata\\df2.csv")
new_file['global_id'] = new_file.players_links.str.extract('(\d+)')
new_file['global_id'] = new_file.global_id.astype('float64')
df_split['global_id'] = pd.to_numeric(df_split['global_id'])

In [114]:
merged = pd.merge(new_file, df_split, how='outer', on='global_id')
merged.drop_duplicates(subset='global_id',inplace=True)

In [115]:
#cleaning up data
def date_clean(date_col):
    x = merged[date_col].str.replace('[":]','')
    return pd.to_datetime(x,errors='coerce')
#create list of column names we want to iterate through
def list_var(col_name):
    return [col_name+str(i) for i in range(1,85)]
#update date columns
var_names = list_var('date')
for i in var_names:
    merged[i] = date_clean(i)

In [116]:
#cleanup first boxer rating
def remove_colon(col_name):
    return merged[col_name].str.replace(':','')
#weights need to be converted to float
def firstweight(col_name):
    a = remove_colon(col_name)
    return pd.to_numeric(a,errors='coerce')
#extract scorecard from judges column
def score_card(col_name):
    return merged[col_name].str.findall('scorecard:(?P<scorecards>.*?)}')
#update first boxer rating columns
boxer_var = list_var('firstBoxerRating')
for i in boxer_var:
    merged[i] = remove_colon(i)
#update weight
weight_var = list_var('firstBoxerWeight')
for i in weight_var:
    merged[i] = firstweight(i)
#extract score card
scorecard_var = list_var('JudgeID')
for i in scorecard_var:
    merged[i] = score_card(i)
#update location
location_var = list_var('location')
for i in location_var:
    merged[i] = remove_colon(i)
#update num of rounds
rounds_var = list_var('numberofrounds')
for i in rounds_var:
    merged[i] = remove_colon(i)
#update outcome
outcome_var = list_var('outcome')
for i in outcome_var:
    merged[i] = remove_colon(i)
#update rating
rating_var = list_var('rating')
for i in rating_var:
    merged[i] = remove_colon(i)

In [131]:
#function creates errors so trying for loop instead
ref_outcome = list_var('referee')
for i in ref_outcome:
    merged[i] = merged[i].str.findall('result:(?P<result>.*?)]')

In [132]:
secondboxer_var = list_var('secondBoxer')
for i in secondboxer_var:
    merged[i] = merged[i].str.extract('name:(?P<name>.*?)}')
secondboxlast_var = list_var('secondBoxerLast6')
for i in secondboxlast_var:
    merged[i] = remove_colon(i)
secondboxrate_var = list_var('secondBoxerRating')
for i in secondboxrate_var:
    merged[i] = remove_colon(i)
secondboxrec_var = list_var('secondBoxerRecord')
for i in secondboxrec_var:
    merged[i] = remove_colon(i)
secondboxweight_var = list_var('secondBoxerWeight')
for i in secondboxweight_var:
    merged[i] = firstweight(i)

In [133]:
#extract time from metadata
def split_time(col):
    return merged[col].str.extract('(\d*\:\d+)',expand=True)
times = list_var('metadata')
for i in times:
    merged[i] = split_time(i)

In [134]:
drop_cols = list(range(0,86))
merged.drop(drop_cols,axis=1,inplace=True)

In [40]:
# create columns with counts of outcomes from referee
# ref_outcomes = list_var('referee')
# merged2 = pd.concat([merged,merged[ref_outcome].astype(str).stack().str.replace('\[|\"','').str.extract('(\w+\s\w+)').groupby(level=0)[0].apply(pd.Series.value_counts).unstack(fill_value=0)],axis=1)

In [137]:
merged.to_csv('C:\\Users\\User\\Documents\\finalvisuals.csv')

In [144]:
path="C:\\Users\\User\\Documents\\GitHub\\SpringboardCapstoneBoxingPredictionWebApp\\boxingdata\\visuals.csv"
path_two= "C:\\Users\\User\\Documents\\finalvisuals.csv"

In [145]:
a = pd.read_csv(path)
b = pd.read_csv(path_two)

C:\Users\User\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (683,686,703,720,737,745,746,748,749,750,751,752,753,754,755,756,757,758,759,761,762,763,765,766,767,768,769,770,771,772,773,774,775,776,778,779,780,782,783,784,785,786,787,788,789,790,791,792,793,795,796,797,799,800,801,802,803,804,805,806,807,808,809,810,812,813,814,816,817,818,819,820,821,822,823,824,825,826,827,829,830,831,833,834,835,836,837,838,839,840,841,842,843,844,846,847,848,850,851,852,853,854,855,856,857,858,859,860,861,863,864,865,867,868,869,870,871,872,873,874,875,876,877,878,880,881,882,884,885,886,887,888,889,890,891,892,893,894,895,897,898,899,901,902,903,904,905,906,907,908,909,910,911,912,914,915,916,918,919,920,921,922,923,924,925,926,927,928,929,931,932,933,935,936,937,938,939,940,941,942,943,944,945,946,948,949,950,952,953,954,955,956,957,958,959,960,961,962,963,965,966,967,969,970,971,972,973,974,975,976,977,978,979,980,982,983,984,986,987,988,989,990,991,992,

In [159]:
viz =pd.concat([a,c],axis=0)
viz.to_csv('C:\\Users\\User\\Documents\\viz.csv')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [149]:
a

,Unnamed: 0,Unnamed: 0.1,miles,location,name,sex,w,l,d,division,...,numberofrounds99,outcome99,rating99,referee99,secondBoxer99,secondBoxerLast699,secondBoxerRating99,secondBoxerRecord99,secondBoxerWeight99,titles99
0,0,0,0.0,USA,Roberto Salas,male,5.0,0.0,0.0,cruiser,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,0.0,USA,James Jackson,male,4.0,0.0,1.0,heavy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6,0.0,USA,Alex Love,female,3.0,0.0,0.0,bantam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,9,0.0,USA,Juan Centeno,male,4.0,3.0,1.0,fly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,12,0.0,USA,Jordan Weeks,male,2.0,0.0,0.0,super middle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3872,3872,12368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3873,3873,12369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3874,3874,12370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3875,3875,12371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
c= b[b['name'] == 'Tyson Fury']
c

,Unnamed: 0,Unnamed: 0.1,miles,location,name,sex,w,l,d,division,...,numberofrounds99,outcome99,rating99,referee99,secondBoxer99,secondBoxerLast699,secondBoxerRating99,secondBoxerRecord99,secondBoxerWeight99,titles99
3842,3842,10735,NaN,"UK, Manchester Lancashire",Tyson Fury,male,29.0,0.0,1.0,heavy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
